# Match sample metadata to NOAA sheet


In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import glob

#For illustrator import:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

### Import Dada2 data

In [4]:
#location of data:
directory = '../data/Dada2_seq_data/'
markers = ['12S']
prefix = 'CN19S'
otus= []
taxas=[]
metas=[]
for marker in markers:
    print('XXXXXX    ',marker)
    df = pd.read_csv(directory+prefix+'_'+marker+'_Dada2_otu_merged.csv')
    df = df.rename(columns={'Unnamed: 0':'ASV'})
    df.set_index('ASV', inplace=True)
    otus.append(df)
    print('Number samples in otu_table:', len(list(df)))
    df = pd.read_csv(directory+prefix+'_'+marker+'_Dada2_meta_merged.csv')
    df.set_index('sample_name', inplace=True)
    #date handling
    print(list(df))
    df['eventDate'] = pd.to_datetime(df['eventDate'])
    df['month'] = df['eventDate'].dt.month
    metas.append(df)
    print('Number samples in metadata table:', len(df.index))
    df = pd.read_csv(directory+prefix+'_'+marker+'_Dada2_taxa_merged.csv')
    df = df.rename(columns={'Unnamed: 0':'ASV'})
    df.set_index('ASV', inplace=True)
    print('Number ASVs in taxa table:', len(df.index))
    taxas.append(df)
taxas[0].head()


XXXXXX     12S
Number samples in otu_table: 286
['FilterID', 'target_gene', 'PlateID', 'library', 'local_time', 'time_label', 'SAMPLING_cruise', 'depth', 'SAMPLING_platform', 'SC', 'ESP', 'SAMPLING_station_number', 'SAMPLING_station', 'SAMPLING_bottle', 'decimalLongitude', 'decimalLatitude', 'sample_type', 'Plates', 'Markers', 'Status', 'Dewar_name', 'Sampling_method', 'replicate', 'SAMPLING_rdepth', 'project_name', 'nitrate', 'fluor', 'density', 'pressure', 'minimumDepthInMeters', 'maximumDepthInMeters', 'start_GMT', 'end_GMT', 'temp', 'salinity', 'sigmat', 'spice', 'diss_oxygen', 'PAR (umol/s/m2)', 'altitude', 'chlorophyll', 'bbp470 (count)', 'bbp650 (count)', 'SAMPLING_project', 'ESP_name', 'diel', 'month', 'day', 'hour', 'eventDate']
Number samples in metadata table: 286
Number ASVs in taxa table: 2827


/var/folders/5v/h4s_0pz17yb6f6krsxkzr5dw000b65/T/ipykernel_21272/1341033839.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['eventDate'] = pd.to_datetime(df['eventDate'])


,Kingdom,Phylum,Class,Order,Family,Genus,Species
ASV,,,,,,,
ASV_1,Eukaryota,Chordata,Actinopteri,Clupeiformes,Engraulidae,Engraulis,Engraulis mordax
ASV_2,Eukaryota,Chordata,Actinopteri,Myctophiformes,Myctophidae,Diaphus,Diaphus theta
ASV_3,Eukaryota,Chordata,Actinopteri,Gadiformes,Macrouridae,unassigned,unassigned
ASV_4,Eukaryota,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius productus
ASV_5,Eukaryota,Chordata,Actinopteri,Myctophiformes,Myctophidae,Stenobrachius,Stenobrachius leucopsarus


### Import NOAA Metadata sheet

- get SRA metadata set up

In [12]:
file = '/Users/kpitz/Projects/NOAA_metadata_conversion/NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
# 6th row has terms
df = pd.read_excel(file, sheet_name = 'study_data', skiprows=5)
print(list(df))
df

['project_id', 'project_name', 'project_id_external', 'project_contact', 'project_description', 'type', 'license', 'citation', 'keywords', 'associated_parties', 'study_area_description', 'external_links', 'recordedBy', 'sampling_description', 'grant_number', 'grant_title', 'grant_agency', 'project_proposal', 'accessions', 'metadata_notes', 'date_modified', 'modified_by']


,project_id,project_name,project_id_external,project_contact,project_description,type,license,citation,keywords,associated_parties,...,recordedBy,sampling_description,grant_number,grant_title,grant_agency,project_proposal,accessions,metadata_notes,date_modified,modified_by
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-10 06:41:52.866,aomlomics@gmail.com


In [9]:
file = '/Users/kpitz/Projects/NOAA_metadata_conversion/NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
# 6th row has terms
df = pd.read_excel(file, sheet_name = 'water_sample_data', skiprows=8)
df

,sample_name,serial_number,cruise_id,line_id,station,ctd_bottle_no,sample_replicate,source_mat_id,biological_replicates,extract_number,...,tot_diss_nitro,tot_inorg_nitro,tot_nitro,tot_part_carb,tot_phosp,turbidity,water_current,description,date_modified,modified_by
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-10 06:41:58.648,aomlomics@gmail.com


In [10]:
file = '/Users/kpitz/Projects/NOAA_metadata_conversion/NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
# 6th row has terms
df = pd.read_excel(file, sheet_name = 'amplicon_prep_data', skiprows=6)
df

,sample_name,library_id,title,library_strategy,library_source,library_selection,lib_layout,platform,instrument_model,design_description,...,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,nucl_acid_amp,adapters,mid_barcode,date_modified,modified_by
0,NaN,NaN,NaN,AMPLICON,GENOMIC,PCR,NaN,ILLUMINA,HiSeq X Ten,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-10 06:42:06.350,aomlomics@gmail.com
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABI_SOLID,AB 5500 Genetic Analyzer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [11]:
file = '/Users/kpitz/Projects/NOAA_metadata_conversion/NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
# 6th row has terms
df = pd.read_excel(file, sheet_name = 'analysis_data', skiprows=6)
df

,amplicon_sequenced,ampliconSize,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,sop,identificationReferences,controls_used,date_modified,modified_by
0,16S V4-V5,411,cutadapt,Tourmaline; qiime2-2021.2; dada2,ASV,Tourmaline; qiime2-2021.2; naive-bayes classifier,Silva SSU Ref NR 99 v138.1; 515f-926r region; ...,https://github.com/aomlomics/gomecc,NaN,10.5281/zenodo.8392695 | https://github.com/ao...,12 distilled water blanks | 2 PCR no-template ...,2024-01-10 06:42:12.057,aomlomics@gmail.com
1,18S V9,260,cutadapt,Tourmaline; qiime2-2021.2; dada2,ASV,Tourmaline; qiime2-2021.2; naive-bayes classifier,PR2 v5.0.1; V9 1391f-1510r region; 10.5281/zen...,https://github.com/aomlomics/gomecc,NaN,10.5281/zenodo.8392706 | https://pr2-database....,12 distilled water blanks | 2 PCR no-template ...,NaT,NaN


In [14]:
# depth bin:
df = metas[0].copy()
df = df[['depth']]
df = df.sort_values('depth')
df = df.loc[df['depth']>=100]
df = df.loc[df['depth']<=300]
bins = [100, 150, 199, 250, 299]
df['binned_depth'] = pd.cut(df['depth'], bins)
df['count'] = 1
df = df.groupby('binned_depth').sum()
df

,depth,count
binned_depth,,
"(100, 150]",4440.3211,34
"(150, 199]",980.1172,6
"(199, 250]",8076.7478,36
"(250, 299]",6529.7768,24


In [ ]:
# sample dates:
